In [1]:
import os
import sys
sys.path.append("/home/jshe/molecular_attention_bias/src")
import toml

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import models
import data

%load_ext autoreload
%autoreload 2

In [2]:
config = toml.load('/home/jshe/molecular_attention_bias/src/config/train/BiasedAttentionTransformer/gaussian_kernel.toml')

output_dir = config['output_dir']
model_config = config['model_config']
dataset_config = config['dataset_config']
train_config = config['train_config']

model_config['out_features'] = len(dataset_config['target_labels'])
model = models.get_model(**model_config)

dataset = data.get_dataset(**dataset_config, split='validation')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)